In [11]:
#importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

%matplotlib inline

In [12]:
df_home = pd.read_csv('home_energy.csv', parse_dates= True, squeeze= True)
df_home['Time'] = pd.to_datetime(df_home['Time'])

In [13]:
df_home = df_home[['Equipment', 'Time', 'Condition']]
df_home.isna().sum()

Equipment    0
Time         0
Condition    0
dtype: int64

In [14]:
df_home.head()

,Equipment,Time,Condition
0,Device 1,2020-02-05 02:03:50,ON
1,Device 1,2020-02-05 05:44:10,ON
2,Device 1,2020-02-05 07:56:38,ON
3,Device 1,2020-02-05 08:55:41,OFF
4,Device 1,2020-02-05 15:15:50,ON


In [15]:
#make a series of the key data points and then index it 
time_serie = pd.Series(df_home['Condition'].values, index= df_home['Time'])
df_home['Hour'] = time_serie.index.hour
df_home['Minute'] = time_serie.index.minute
df_home['Seconds'] = time_serie.index.second

time_serie.head()

Time
2020-02-05 02:03:50     ON
2020-02-05 05:44:10     ON
2020-02-05 07:56:38     ON
2020-02-05 08:55:41    OFF
2020-02-05 15:15:50     ON
dtype: object

In [16]:
df_home.head()

,Equipment,Time,Condition,Hour,Minute,Seconds
0,Device 1,2020-02-05 02:03:50,ON,2,3,50
1,Device 1,2020-02-05 05:44:10,ON,5,44,10
2,Device 1,2020-02-05 07:56:38,ON,7,56,38
3,Device 1,2020-02-05 08:55:41,OFF,8,55,41
4,Device 1,2020-02-05 15:15:50,ON,15,15,50


In [17]:
df_home = pd.get_dummies(df_home, columns=['Equipment'], prefix=['Equipment'])

In [18]:
df_home.head()

,Time,Condition,Hour,Minute,Seconds,Equipment_Device 1,Equipment_Device 2,Equipment_Device 3,Equipment_Device 4
0,2020-02-05 02:03:50,ON,2,3,50,1,0,0,0
1,2020-02-05 05:44:10,ON,5,44,10,1,0,0,0
2,2020-02-05 07:56:38,ON,7,56,38,1,0,0,0
3,2020-02-05 08:55:41,OFF,8,55,41,1,0,0,0
4,2020-02-05 15:15:50,ON,15,15,50,1,0,0,0


In [28]:
y = df_home['Condition'].replace(['ON', 'OFF'], [0,1])
x = df_home.drop(axis= 1, columns= ['Condition', 'Time'])

In [29]:
x.head()

,Hour,Minute,Seconds,Equipment_Device 1,Equipment_Device 2,Equipment_Device 3,Equipment_Device 4
0,2,3,50,1,0,0,0
1,5,44,10,1,0,0,0
2,7,56,38,1,0,0,0
3,8,55,41,1,0,0,0
4,15,15,50,1,0,0,0


In [30]:
y.value_counts()

0    275
1    239
Name: Condition, dtype: int64

In [34]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=7,test_size=0.2)
classifier = RandomForestClassifier(random_state=42)

In [35]:
classifier.fit(x_train, y_train)
y_preds = classifier.predict(x_test)

print('The accuracy score is', accuracy_score(y_test, y_preds))

The accuracy score is 0.6796116504854369


In [36]:
print(classifier.get_params)

<bound method BaseEstimator.get_params of RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)>


In [ ]:
#implementing the a